# **Project 4: LLM Project Activity - Topic Modeling**




### **Week 23** Project 4: 4-optimization
* Evaluating and optimizing the project model and creating an LLM Model Report. (9.5)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
 !apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!pip install -q transformers datasets evaluate

In [ ]:
repo_name = "newsgroups_model"  #model repo name

In [ ]:
#train model, load DistilBert
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased",
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#define training hyperparameters in TrainingArguments, choose appropriate metric for model to calculate metrics during training, pass training arguments to Trainer, call train() to finetune model;F1 will determine best model at end
#metrics selected: accuracy overall percentage of correctly classified documents and Macro F1 Score to determine mean of precision/recall per class and averaged
import numpy as np
from evaluate import load

# Load all required metrics
accuracy = load("accuracy")
f1 = load("f1")
precision = load("precision")
recall = load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "precision": precision.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="macro")["recall"],
    }

In [ ]:
#model #1: optimize through hyperparameters tuning of reducing per device train batch size (from 16 to 8 to see if able to generlize better on small dataset) and per device eval bath size (reduced to 8, doesn't impact mdoel quality but will impact evaluation speed; to match train batch size), added weight decay to prevent overfitting, and warm up steps to see if increase learning rate
repo_name = "newsgroups_model"
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=1e-5,  #lower learning rate smaller batch size
   per_device_train_batch_size=8,
   per_device_eval_batch_size=8,
   gradient_accumulation_steps=2,  #simulates batch size of 16
   num_train_epochs=2,
   weight_decay=0.01,
   warmup_ratio=0.1,

   eval_strategy="epoch",
   save_strategy="epoch",

   push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alia-locken (alia-locken-lighthouse-labs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision,Recall
1,2.319700,1.424123,0.643521,0.618610,0.635969,0.624553
2,1.379700,1.233425,0.663834,0.643437,0.647780,0.645893


TrainOutput(global_step=1416, training_loss=1.6558053399209922, metrics={'train_runtime': 1338.7347, 'train_samples_per_second': 16.903, 'train_steps_per_second': 1.058, 'total_flos': 2998434498723840.0, 'train_loss': 1.6558053399209922, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.2334253787994385,
 'eval_accuracy': 0.6638343069569835,
 'eval_f1_macro': 0.6434372248870809,
 'eval_precision': 0.647779574786804,
 'eval_recall': 0.6458928223813728,
 'eval_runtime': 113.7454,
 'eval_samples_per_second': 66.218,
 'eval_steps_per_second': 8.282,
 'epoch': 2.0}

**Outputs Summary:** from epoch 1 to 2, training loss decreased significantly (2.31 to 1.37), validation loss decreased (1.42 to 1.23), and all metrics (accuracy, F1, precision, recall) improved. Model appears to be learning well and generalizing better. The loss and metrics indicate good overall learning progress. Will look to add one additional epoch (increasing from 2 to 3) to see if the model will further improve performance.

In [ ]:
#model #2: optimize using same hyperparameters but adding in addition epoch to see if performance improvments observed with 3 versus 2
repo_name = "newsgroups_model"
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=1e-5,  #lower learning rate smaller batch size
   per_device_train_batch_size=8,
   per_device_eval_batch_size=8,
   gradient_accumulation_steps=2,  #simulates batch size of 16
   num_train_epochs=3,
   weight_decay=0.01,
   warmup_ratio=0.1,

   eval_strategy="epoch",
   save_strategy="epoch",

   push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision,Recall
1,1.033700,1.070870,0.687069,0.662192,0.660909,0.669988
2,0.797200,1.056222,0.688263,0.666567,0.676877,0.670685
3,0.648300,1.033279,0.696628,0.674041,0.679325,0.679723


TrainOutput(global_step=2124, training_loss=0.7969629787006846, metrics={'train_runtime': 2016.4573, 'train_samples_per_second': 16.832, 'train_steps_per_second': 1.053, 'total_flos': 4497651748085760.0, 'train_loss': 0.7969629787006846, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.0332790613174438,
 'eval_accuracy': 0.6966277217206586,
 'eval_f1_macro': 0.6740413540599004,
 'eval_precision': 0.6793250251214719,
 'eval_recall': 0.6797230401635634,
 'eval_runtime': 113.7029,
 'eval_samples_per_second': 66.243,
 'eval_steps_per_second': 8.285,
 'epoch': 3.0}

**Outputs Summary:** from epoch 1 to 3, training loss decreased significantly (1.03 to 0.64), validation loss decreased (1.07 to 1.03), and all metrics (accuracy, F1, precision, recall) improved. Model appears to be learning well and generalizing better. The loss and metrics indicate good overall learning progress and it appears adding the additional epoch, and subsequent model outputs, support this conclusion.

In [ ]:
#outputs of model #2 are results pushing to Hugging Face Hub for later use
trainer.push_to_hub()

Uploading...:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alocken/newsgroups_model/commit/589a3b34fe845823c367b6f4984c9537791daf79', commit_message='End of training', commit_description='', oid='589a3b34fe845823c367b6f4984c9537791daf79', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alocken/newsgroups_model', endpoint='https://huggingface.co', repo_type='model', repo_id='alocken/newsgroups_model'), pr_revision=None, pr_num=None)

In [5]:
#use model to predict on new text using pipelines module

from transformers import pipeline
data = [
    "The Hubble Space Telescope captured a new image of a galaxy.",
    "I need help setting up my Linux dual boot system.",
    "The new graphics card performs exceptionally in gaming benchmarks.",
    "The Lakers won their game last night against the Bulls."
]
my_model = pipeline(model="alocken/newsgroups_model")
my_model(data)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'sci.space', 'score': 0.878742516040802},
 {'label': 'comp.sys.ibm.pc.hardware', 'score': 0.45447736978530884},
 {'label': 'comp.sys.mac.hardware', 'score': 0.5598772168159485},
 {'label': 'rec.sport.hockey', 'score': 0.9507641196250916}]

In [7]:
#Hugging Face Hub
classifier = pipeline("text-classification", model="alocken/newsgroups_model")

# Run predictions
predictions = classifier(data)
print(predictions)

Device set to use cpu


[{'label': 'sci.space', 'score': 0.878742516040802}, {'label': 'comp.sys.ibm.pc.hardware', 'score': 0.45447736978530884}, {'label': 'comp.sys.mac.hardware', 'score': 0.5598772168159485}, {'label': 'rec.sport.hockey', 'score': 0.9507641196250916}]


In [8]:
for text, pred in zip(data, predictions):
    print(f"Text: {text}")
    print(f"Predicted Label: {pred['label']}, Confidence: {pred['score']:.2f}")
    print()

Text: The Hubble Space Telescope captured a new image of a galaxy.
Predicted Label: sci.space, Confidence: 0.88

Text: I need help setting up my Linux dual boot system.
Predicted Label: comp.sys.ibm.pc.hardware, Confidence: 0.45

Text: The new graphics card performs exceptionally in gaming benchmarks.
Predicted Label: comp.sys.mac.hardware, Confidence: 0.56

Text: The Lakers won their game last night against the Bulls.
Predicted Label: rec.sport.hockey, Confidence: 0.95



**Evaluating performance of the model on unseen data outputs summary:** the output demonstrates that the model is working correctly and making predictions on the custom data input. Output corresponds to one input sentence from data list, label provides the predicted class, and the score is the model's confidence in the prediction (range 0 to 1).
Interpreting the results, it looks like the Hubble Space text received high confidence (0.88) and was correctly interpreted, confidently, that it's related to science/space. In the Linux and gaming benchmarks text, Linux received low confidence and gaming moderate. It appears that there was some uncertainty around how to relate For the Lakers text, it was very confident (0.95) although it was correct in it being a rec sport, it incorrectly predicted hockey instead of basketball. This indicates a false positive.
Overall, the model performs well when topics are distinct and demonstrates much lower confidence when the topic is either overlapping or not distinct (unclear or ambiguous). It can misclassify similar domains as well which could be due to fewer examples in one class or class imbalance.
To further improve this model’s performance, a few ideas could be:
- fine-tune longer with enhanced hyperparameters
- add additional training examples for any underperforming categories

Note: below code is notebook cleaning code to support upload to GitHub from Google Colab.

In [1]:
from google.colab import drive
import IPython

notebook_filename = "Project_4_4_optimization.ipynb"

# Save current notebook to file system
IPython.get_ipython().run_cell_magic('capture', '', f'%%javascript\nIPython.notebook.save_checkpoint();')
print(f"Saved current notebook to: {notebook_filename}")

Saved current notebook to: Project_4_4_optimization.ipynb


In [2]:
# upload it from your local drive
from google.colab import files
uploaded = files.upload()

Saving Project_4_4_optimization.ipynb to Project_4_4_optimization.ipynb


In [4]:
import nbformat

notebook_filename = "Project_4_4_optimization.ipynb"  # Make sure this matches the uploaded filename
output_filename = notebook_filename.replace('.ipynb', '_clean.ipynb')

# Load the notebook
with open(notebook_filename, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

# Clean metadata
for cell in nb.cells:
    cell.metadata.pop('colab', None)
    cell.metadata.pop('widgets', None)
nb.metadata.pop('colab', None)
nb.metadata.pop('widgets', None)

# Save cleaned version
with open(output_filename, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print(f"Cleaned notebook saved as: {output_filename}")

Cleaned notebook saved as: Project_4_4_optimization_clean.ipynb
